In [1]:
import ee
import geemap
import fiona

In [2]:
Map = geemap.Map()
Map

  

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# Center it over London
Map.setCenter(-0.0894, 51.4964, 9.5) # center on London (lat, lon, zoom)


In [4]:
# Load the Municipal Areas (FeatureCollection)
london_extent_shp = '../data/shp/london_by_ward/raw_shp_file/LondonWard.shp'
london_wards = geemap.shp_to_ee(london_extent_shp)

Map.addLayer(london_wards, {}, 'London Wards')

# NOTE:  This will create a warning on Windows. I'm successfully ignoring it for now.
#        "FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas 
#        in a future version. Use pandas.Index with the appropriate dtype instead. 
#        pd.Int64Index"


C:\Users\paulw\miniconda3\envs\gee\envs\geemap\envs\gee2\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [5]:
# LANDCOVER
# Not sure if this will be significant. There isn't much in London. 
#
# NOTE: See that `clipToCollection` method? It chops up our Rasters using our feature collection.
#
#    @see https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1#bands
#    @see Clipping to a FeatureCollection https://developers.google.com/earth-engine/apidocs/ee-image-clip
#
landcover = ee.Image('MODIS/051/MCD12Q1/2013_01_01').select('Land_Cover_Type_1').clipToCollection(london_wards)
Map.addLayer(landcover, {}, 'MODIS Land Cover')



In [6]:
# @see https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1#bands
landcover2Col = ee.Image('MODIS/051/MCD12Q1/2001_01_01').select('Land_Cover_Type_1')#.clipToCollection(london_wards)
#landcover2Image = landcover2Col.first()
Map.addLayer(landcover2Col, {}, 'MODIS Land Cover 2019-12-01')


In [7]:
# @see https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1#bands
MODIS_VIS_PALLETTE = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

# @see Clipping to a FeatureCollection https://developers.google.com/earth-engine/apidocs/ee-image-clip
modisLandcoverCol = ee.ImageCollection('MODIS/006/MCD12Q1').select('LC_Type1')
#
# NOTE: See that `clipToCollection` method? It chops up our Rasters using our feature collection.
#
# early years
landcover2Image = modisLandcoverCol.filterDate('2019-01-01', '2019-12-31').first().clipToCollection(london_wards)
Map.addLayer(landcover2Image, MODIS_VIS_PALLETTE, 'MODIS 2019')

# later years
landcover2Image = modisLandcoverCol.filterDate('2001-01-01', '2001-01-02').first().clipToCollection(london_wards)
Map.addLayer(landcover2Image, MODIS_VIS_PALLETTE, 'MODIS 2001')

In [8]:
# check to see what kind of data type this is so we can see what we can do with it.
london_wards

In [9]:
ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(legend_title="MODIS Global Land Cover", legend_dict=legend_dict)



In [16]:
greennessVis = {
  'min': 0.1,
  'max': 0.85,
  'palette': ['fffdcd', 'e1cd73', 'aaac20', '5f920c', '187328', '144b2a', '172313']
};

# Visualization parameters for true color display.
trueColorVis = {
  'min': 0.0,
  'max': 0.4,
  'gamma': 1.2,
};

    
    
def greenessPixel( startDate, endDate, layerTitle ):
    
    """
    Method for making layers with Greenest NDVI pixel: 
    Possible dates: 1999-01-01T00:00:00Z - 2021-01-01T00:00:00
    
    Parameters
    ----------
    startDate : str
        Date start string. Like '2002-01-01'
    endDate : str
        Date end string. Like '2002-12-31'
    layerTitle : str
        The title prefix that will differentiate the two layers it will create from others.
        
    @see https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_ANNUAL_GREENEST_TOA?hl=en
    """

    # Import the dataset, filter to 2002, get the image.

    dataset = ee.ImageCollection('LANDSAT/LE07/C01/T1_ANNUAL_GREENEST_TOA').filterDate(startDate, endDate).first();

    # Get the greenness band.
    greenness = dataset.select('greenness').clipToCollection(london_wards);

    # Get bands for true color RGB display.
    trueColor = dataset.select(['B3', 'B2', 'B1']).clipToCollection(london_wards); # NOTE: See that `clipToCollection` method? It chops up our Rasters using our feature collection.

    # Define a mask for low greenness, make low greenness partially transparent.
    mask = dataset.select('greenness').gt(0.1).remap([0, 1], [0.5, 1]);

    # Visualization parameters for greenness display.
    
    maskedGreeness = greenness.updateMask(mask)
    
    # Display the layers on the map with mask applied.
    Map.addLayer(trueColor.updateMask(mask), trueColorVis, layerTitle + ' True Color (432)', False);
    Map.addLayer(maskedGreeness, greennessVis, layerTitle + ' Greenness');
    
    return maskedGreeness

print("Starting...")
greenessByWard_2011_img = greenessPixel('2011-01-01', '2011-12-31', "2011")
print("2011 complete")
greenessByWard_2012_img = greenessPixel('2012-01-01', '2012-12-31', "2012")
print("2012 complete")
greenessByWard_2013_img = greenessPixel('2013-01-01', '2013-12-31', "2013")
print("2013 complete")
greenessByWard_2014_img = greenessPixel('2014-01-01', '2014-12-31', "2014")
print("2014 complete")
greenessByWard_2015_img = greenessPixel('2015-01-01', '2015-12-31', "2015")
print("2015 complete")
greenessByWard_2016_img = greenessPixel('2016-01-01', '2016-12-31', "2016")
print("2016 complete")
greenessByWard_2017_img = greenessPixel('2017-01-01', '2017-12-31', "2017")
print("2017 complete")
greenessByWard_2018_img = greenessPixel('2018-01-01', '2018-12-31', "2018")
print("2018 complete")
greenessByWard_2019_img = greenessPixel('2019-01-01', '2019-12-31', "2019")
print("2019 complete")
greenessByWard_2020_img = greenessPixel('2020-01-01', '2020-12-31', "2020")
print("2020 complete")
greenessByWard_2021_img = greenessPixel('2021-01-01', '2021-12-31', "2021")
print("2021 complete")


Starting...
2011 complete
2012 complete
2013 complete
2014 complete
2015 complete
2016 complete
2017 complete
2018 complete
2019 complete
2020 complete
2021 complete


EEException: Image.select: Parameter 'input' is required.

In [17]:
import os
import ee
import geemap

def createShpWithMeanGreen( img, fc ):
    # try and get the average "Greeness" (aka NDVI)
    # feature collection
    return img.reduceRegions( fc, ee.Reducer.mean(), 30, "EPSG:32629"); # london EPSG

greenessByWard_2011 = createShpWithMeanGreen(greenessByWard_2011_img, london_wards)
greenessByWard_2012 = createShpWithMeanGreen(greenessByWard_2012_img, london_wards)
greenessByWard_2013 = createShpWithMeanGreen(greenessByWard_2013_img, london_wards)
greenessByWard_2014 = createShpWithMeanGreen(greenessByWard_2014_img, london_wards)
greenessByWard_2015 = createShpWithMeanGreen(greenessByWard_2015_img, london_wards)
greenessByWard_2016 = createShpWithMeanGreen(greenessByWard_2016_img, london_wards)
greenessByWard_2017 = createShpWithMeanGreen(greenessByWard_2017_img, london_wards)
greenessByWard_2018 = createShpWithMeanGreen(greenessByWard_2018_img, london_wards)
greenessByWard_2019 = createShpWithMeanGreen(greenessByWard_2019_img, london_wards)
greenessByWard_2020 = createShpWithMeanGreen(greenessByWard_2020_img, london_wards)
greenessByWard_2021 = createShpWithMeanGreen(greenessByWard_2021_img, london_wards)


In [18]:
def writeToShpFile( fc, fileName):
    # Write out shape file
    out_dir = os.path.expanduser("~/Downloads")

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    out_shp = os.path.join(out_dir, fileName)
    geemap.ee_export_vector(fc, out_shp, verbose=True)

writeToShpFile( greenessByWard_2011, "London_MeanGreenessByWard_2011.shp")
writeToShpFile( greenessByWard_2012, "London_MeanGreenessByWard_2012.shp")
writeToShpFile( greenessByWard_2013, "London_MeanGreenessByWard_2013.shp")
writeToShpFile( greenessByWard_2014, "London_MeanGreenessByWard_2014.shp")
writeToShpFile( greenessByWard_2015, "London_MeanGreenessByWard_2015.shp")
writeToShpFile( greenessByWard_2016, "London_MeanGreenessByWard_2016.shp")
writeToShpFile( greenessByWard_2017, "London_MeanGreenessByWard_2017.shp")
writeToShpFile( greenessByWard_2018, "London_MeanGreenessByWard_2018.shp")
writeToShpFile( greenessByWard_2019, "London_MeanGreenessByWard_2019.shp")
writeToShpFile( greenessByWard_2020, "London_MeanGreenessByWard_2020.shp")
writeToShpFile( greenessByWard_2021, "London_MeanGreenessByWard_2021.shp")


Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2011.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2012.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2013.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2014.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2015.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2016.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2017.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_MeanGreenessByWard_2018.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\paulw\Downloads\London_Me